# Journal de bord

Copier-coller les cellules comme canevas pour y remplir les informations.

## 📅 Journal de bord – Jour 1 (17/12/2025)

### 🔧 Étapes / Actions réalisées
- Découverte du sujet,
- Récupération des ressources,
- Structuration des dossiers
- Mise en place du tracking **git**

**Première approche de découpage des tâches**

**Traitement des fichiers de données tabulaires**
1. Analyser les fichiers de données et définir la stratégie de nettoyage préalable : données sensibles, données personnelles, données interdites,
2. Procéder à la fusion des jeux de données en incluant une information supplémentaire pour ne pas perdre l'origine de l'information
3. Produire dans outputs/data_processed le fichier csv résultant

### 📝 Observations / Difficultés
- 

### ✅ Prochaines étapes
- 